<a href="https://colab.research.google.com/github/PnrSvc/Bert-Examples/blob/main/example_tr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch==1.10.2+cu113 torchvision==0.11.3+cu113 torchaudio===0.10.2+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
!pip install transformers requests beautifulsoup4 pandas numpy
from transformers import AutoTokenizer,AutoModelForSequenceClassification,pipeline
import torch
import requests
from bs4 import BeautifulSoup
import re 
import numpy as np
import pandas as pd

Looking in links: https://download.pytorch.org/whl/cu113/torch_stable.html
     |██████████████▋                 | 834.1 MB 1.4 MB/s eta 0:11:49tcmalloc: large alloc 1147494400 bytes == 0x5646d23d4000 @  0x7fb08f58d615 0x564698fa53bc 0x56469908618a 0x564698fa81cd 0x56469909ab3d 0x56469901c458 0x56469901702f 0x564698fa9aba 0x56469901c2c0 0x56469901702f 0x564698fa9aba 0x564699018cd4 0x56469909b986 0x564699018350 0x56469909b986 0x564699018350 0x56469909b986 0x564699018350 0x564698fa9f19 0x564698feda79 0x564698fa8b32 0x56469901c1dd 0x56469901702f 0x564698fa9aba 0x564699018cd4 0x56469901702f 0x564698fa9aba 0x564699017eae 0x564698fa99da 0x564699018108 0x56469901702f
     |██████████████████▌             | 1055.7 MB 1.3 MB/s eta 0:09:31tcmalloc: large alloc 1434370048 bytes == 0x564716a2a000 @  0x7fb08f58d615 0x564698fa53bc 0x56469908618a 0x564698fa81cd 0x56469909ab3d 0x56469901c458 0x56469901702f 0x564698fa9aba 0x56469901c2c0 0x56469901702f 0x564698fa9aba 0x564699018cd4 0x56469909b986 0x5646

In [2]:
tokenizer=AutoTokenizer.from_pretrained('savasy/bert-base-turkish-sentiment-cased')
model=AutoModelForSequenceClassification.from_pretrained('savasy/bert-base-turkish-sentiment-cased')
turk=pipeline("sentiment-analysis",tokenizer=tokenizer,model=model)

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/596 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/422M [00:00<?, ?B/s]

In [3]:
r=requests.get('https://www.gittigidiyor.com/elektrikli-mutfak-aletleri/fritoz/arcelik-k-1799-beyaz/yorumlari')
soup=BeautifulSoup(r.text,'html.parser')
regex=re.compile('.*user-catalog-review-comment-detail gg-d-23 gg-m-22 pl0.*')
result=soup.find_all('div',{'class':regex})
reviews=[result.text.strip() for result in result]

In [4]:
reviews

['Mükemmel bir makine, kızartma derdinden kurtulmuş oldum, patatesleri çıtır öıtır yapıyor ve yumuşak olmuyor, sebze ve  tüm et ürünlerini istediğim gibi kızartabiliyorum ve yumuşak olmuyor, lezzetli ve yağ çekmeden harika kızartmalar yapıyor, kullanımı kolay ve çıkarılabilir tencere sayesinde',
 'eşim kızartmayı çok seviyo. Tencere de ya da tavada yapınca saga sola çok yağ sıçrıyo. Fritözü aldıktan sonra hem yağ sıçrama sorunundan kurtuldum. Hem de fritözler baya az yağ tüketiyomuş onu anladım. Metal sepetinin temizlenmesi de çok rahat. Kısa sürede her türlü ürünün kızartmasını yapabiliyor. kullanımı rahat ve pratik.',
 'Annemin çok eskiden kalma bir fritözü vardı. Çocukluğumdan beri kullanıyor öyle söyleyeyim. Eski modelelrdendi. Ben de anneler günün de annem bu yeni çıkan fritözlerden alayım dedim. Arçelikten Arçelik K 1799 bu model fritözüaldım. Çok az yağla bir sürü kızartma yapıyorsunuz hakikaten. Ben de kullanıyorum annem de kullanıyor. Kullanımı oldukça basit. Temizliği gayety 

In [6]:
df=pd.DataFrame(np.array(reviews), columns=['review'])

In [7]:
import nltk
nltk.download('punkt')
df["tokenized_sents"] = df.apply(lambda row: nltk.sent_tokenize(row["review"]), axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
df.head()

,review,tokenized_sents
0,"Mükemmel bir makine, kızartma derdinden kurtul...","[Mükemmel bir makine, kızartma derdinden kurtu..."
1,eşim kızartmayı çok seviyo. Tencere de ya da t...,"[eşim kızartmayı çok seviyo., Tencere de ya da..."
2,Annemin çok eskiden kalma bir fritözü vardı. Ç...,"[Annemin çok eskiden kalma bir fritözü vardı.,..."
3,Çeyizimde fritözüm olsun olmasın mı biraz tere...,[Çeyizimde fritözüm olsun olmasın mı biraz ter...
4,Olumlu bir yönünü tecrübe edemedim.,[Olumlu bir yönünü tecrübe edemedim.]


In [9]:
for x in df['tokenized_sents']:
  p=turk(x)
  print(p)

[{'label': 'positive', 'score': 0.9849374890327454}]
[{'label': 'negative', 'score': 0.95003741979599}, {'label': 'negative', 'score': 0.9985426664352417}, {'label': 'negative', 'score': 0.9965261816978455}, {'label': 'negative', 'score': 0.770180344581604}, {'label': 'negative', 'score': 0.8653866052627563}, {'label': 'positive', 'score': 0.9123502373695374}, {'label': 'negative', 'score': 0.923634946346283}]
[{'label': 'positive', 'score': 0.8353760838508606}, {'label': 'positive', 'score': 0.9970267415046692}, {'label': 'negative', 'score': 0.9991183876991272}, {'label': 'positive', 'score': 0.8980759978294373}, {'label': 'positive', 'score': 0.7222687005996704}, {'label': 'negative', 'score': 0.9952287673950195}, {'label': 'positive', 'score': 0.9122858643531799}, {'label': 'positive', 'score': 0.6561024785041809}, {'label': 'positive', 'score': 0.7020564675331116}]
[{'label': 'negative', 'score': 0.9093195199966431}, {'label': 'negative', 'score': 0.9854635000228882}, {'label': 'p

In [12]:
df['tokenized_sents'].iloc[1]

['eşim kızartmayı çok seviyo.',
 'Tencere de ya da tavada yapınca saga sola çok yağ sıçrıyo.',
 'Fritözü aldıktan sonra hem yağ sıçrama sorunundan kurtuldum.',
 'Hem de fritözler baya az yağ tüketiyomuş onu anladım.',
 'Metal sepetinin temizlenmesi de çok rahat.',
 'Kısa sürede her türlü ürünün kızartmasını yapabiliyor.',
 'kullanımı rahat ve pratik.']

In [13]:
P=turk('ürün muhteşem')
print(P)

[{'label': 'positive', 'score': 0.9904757142066956}]
